# **Exercício de Estimação de Demanda**

### Apresentação

Esse exemplo é baseado na 12ª edição do livro **Managerial Economics - Foundations of Business Analysis and Strategy** de Thomas e Maurice (2016).

Os dados utilizados estão no diretório, no arquivo `Data for Checckers Pizza.xlsx`.

Nesse exercício, a pizzaria Chechers Pizza quer estabelecer seu preço. Para isso, é muito útil estimar a demanda de pizza da Checkers Pizza. 
<br>A gerente usa a seguinte especificação linear para estimar sua demanda usando 24 observações mensais que ela coletou:

---
### Modelo

$$ 
Q = a + bP + cM + dP_{Al} + eP_{BMac} 
$$

Where

$Q =$ quantidade de pizzas vendidas pela Checkers Pizza

$P = $ Preço da pizza na Checkers Pizza

$M = $ renda média anual dos consumidores em Westbury

$P_{Al} = $ preço da pizza no Al's Pizza Oven (concorrente)

$P_{BMac} = $ preço do Big Mac no McDonald's

In [1]:
import pandas as pd
from statsmodels.formula.api import ols

#### Dados

In [2]:
data_market = pd.read_excel('Data for Checkers Pizza.xlsx')
data_market.head()

,Observation,Q,P,M,P_Al,P_BMac
0,1,2659,8.65,25500,10.55,1.25
1,2,2870,8.65,25600,10.45,1.35
2,3,2875,8.65,25700,10.35,1.55
3,4,2849,8.65,25970,10.30,1.05
4,5,2842,8.65,25970,10.30,0.95


#### Regressão

Nesse caso, nós podemos suprimir a especificação da variável `family`, porque o _defaut_ `Gaussian`.

In [3]:
model_formula = 'Q ~ P + M + P_Al + P_BMac'

model_fit = ols(formula=model_formula,
                data=data_market).fit()

print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      Q   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     101.9
Date:                Thu, 14 Dec 2023   Prob (F-statistic):           1.47e-12
Time:                        16:44:27   Log-Likelihood:                -121.19
No. Observations:                  24   AIC:                             252.4
Df Residuals:                      19   BIC:                             258.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1183.8025    506.298      2.338      0.0

Intervalos de confiança

In [4]:
print(model_fit.conf_int(alpha=0.05))

                    0            1
Intercept  124.108581  2243.496392
P         -241.649087  -185.194716
M            0.065114     0.117063
P_Al        20.202862   182.402851
P_BMac      15.124471   128.565127


Essa é uma parte muito importante na análise de regressão. 

A verificação dos intervalos de confiança nos dá um _range_ de efeitos possíveis de cada uma das variáveis explicativas.
<br>Por exemplo, o coeficiente do preço foi estimado em `-213.4219`. 
<br>Entretanto, como dizia um saudoso mestre, se você quiser 95% de acertar seu _"chute"_, precisa considerar `-241.649087` como limite inferior e `-185.194716` como limite superior.

Isso deve ser considerado em todas as variáveis estimadas.
Contudo, para fins de simplicidade (didática), seguiremos para as análises sem aplicar os intervalos de confiança. 

---
### Algumas análises

#### Quantidades

A gerente decide calcular as elasticidades estimadas da demanda, usando o valor das variáveis que tipificam o mercado.
<br> Esses valores, _para ela_, são os **valores médios de cada variável**. Aqui sinaliza-se esse valores com uma *barra*.

No ponto "típico" da demanda, podemos determinar a quantidade demanda.

$$ \hat{Q} = \hat{a} + \hat{b}\bar{P} + \hat{c}\bar{M} + \hat{d}\bar{P}_{Al} + \hat{e}\bar{P}_{BMac} $$

Para isso, usamos o método `.predict()`.

Como _inputs_, usamos a média das variáveis direto do DataFrame. Dê uma olhada!

In [5]:
inputs = data_market.mean()
print(inputs)

Observation       12.500000
Q               2784.666667
P                  9.046250
M              26613.208333
P_Al              10.118750
P_BMac             1.145833
dtype: float64


In [6]:
# Estimated average quantity
Q_hat =  model_fit.predict(data_market.mean())[0]
print('Q_hat = ',Q_hat)
print('Para esses valores, a quantidade demandada para o Checker Pizza é, aproximadamente, {:.0f}.'.format(Q_hat))

Q_hat =  2784.66666666077
Para esses valores, a quantidade demandada para o Checker Pizza é, aproximadamente, 2785.


#### Elasticidades

Agora, estimamos as elasticidades como segue:

$$ \hat{E} = \hat{b}(\bar{P}/\hat{Q}) $$

$$ \hat{E}_M = \hat{c}(\bar{M}/\hat{Q}) $$

$$ \hat{E}_{XAl} = \hat{d}(\bar{P}_{Al}/\hat{Q}) $$

$$ \hat{E}_{XBMac} = \hat{e}(\bar{P}_{BMac}/\hat{Q}) $$

Where,

$\hat{E} = $ elasticidade-preço da demanda

$\hat{E}_M = $ Elasticidade da renda

$\hat{E}_{XAl} = $ Elasticidade-preço cruzada entre o preço do Al e quantidade do Checkers

$\hat{E}_{XBMac} = $ Elasticidade-preço cruzada entre o preço do Big Mac's e a quantidade do Checkers

In [7]:
# Coefficients
fit = model_fit.params.loc
a_hat = fit['Intercept'] 
b_hat = fit['P']
c_hat = fit['M']
d_hat = fit['P_Al']
e_hat = fit['P_BMac']
# Average value of variables
mean = data_market.describe().loc['mean'].loc
P_bar = mean['P']
M_bar = mean['M']
P_Al_bar = mean['P_Al']
P_BMac_bar = mean['P_BMac']

# Elasticities
E = b_hat*(P_bar/Q_hat)
E_M = c_hat*(M_bar/Q_hat)
E_XAl = d_hat*(P_Al_bar/Q_hat)
E_XBMac = e_hat*(P_BMac_bar/Q_hat)

print('E = ',E,
      '\nE_M = ',E_M,
      '\nE_XAl = ',E_XAl,
      '\nE_XBMac = ',E_XBMac)

E =  -0.6933209981256101 
E_M =  0.8705356509822103 
E_XAl =  0.36810807211162117 
E_XBMac =  0.029562664219400972


Os resultados nos dizem algo interessante:

- A esse preço, a <font color='blue'>_**demanda é inelástica**_</font>. Em outras palavras, se o preço varia, a mudança percentual na demanda será menor que a observada no preço.

- Um aumento de 10% na renda média dos consumidores causa uma elevação da vendas em 8.71% - as pizzas são um <font color='blue'>_**bem normal**_</font>.

- Se a preço da Pizza do Al sobe em 10%, as vendas do Checkers' Pizza aumentam em 3.68%.

- O efeito do preço do Big Mac nas vendas do Checkers' Pizza é bem pequeno: se o preço do Big Mac cai em 10%, as vendas do Checkers' Pizza caem em 0.29%.

---
### Considerações finais

É possível aplicar essa análise de quantidade e elasticidade em diferentes pontos ao longo da curva de demanda.

Também é importante lembrar que este é um exercício de estimação para a demanda de uma firma tomadora de preços, que é consideravelmente mais simples que estimar a demanda de toda uma indústria.

Além disso, existem algumas precauções necessárias quando estes métodos de estimação, como atenção à correta especificação do modelo e as consideraçõoes sobre os intervalos de confiação.

Of course, in the regression literature, there are many other interesting aspects of these methods and other relevant points about the care required when using them.

I hope it help you!